# Workgroup - group 3

## **Math Demonstrations**

### 1. Prove the Frisch-Waugh-Lovell theorem

Given the model:
\begin{align*}
y &= D \beta_1 + W \beta_2 + \mu
\end{align*}
where $y$ is an $n \times 1$ vector, $D$ is an $n \times k_1$ matrix, $\beta_1$ is a $k_1 \times 1$ vector, $W$ is an $n \times k_2$ matrix, $\beta_2$ is a $k_2 \times 1$ vector, and $\mu$ is an $n \times 1$ vector of error terms.

We can construct the following equation:

\begin{align*}
\epsilon_y &= \epsilon_D \phi + \xi
\end{align*}

Running $y$ on $W$, we get:

\begin{align*}
y &= W\hat{\alpha}_1 + \epsilon_y \iff \epsilon_y &= y - W\hat{\alpha}_1
\end{align*}

Similarly, running $D$ on $W$ gives us:

\begin{align*}
D &= W\hat{\alpha}_2 + \epsilon_D \iff \epsilon_D &= D - W\hat{\alpha}_2
\end{align*}

Running $\epsilon_y$ on $\epsilon_D$:
\begin{align*}
y - W \hat{\alpha}_1 &= (D - W \hat{\alpha}_2) \phi + \xi \\
y &= W \hat{\alpha}_1 + (D - W \hat{\alpha}_2) \phi + \xi \\
y &= W \hat{\alpha}_1 + D \phi - W \hat{\alpha}_2 \phi + \xi \\
y &= D \phi + W (\hat{\alpha}_1 - \hat{\alpha}_2 \phi) + \xi
\end{align*}

Comparing the original model with this, we can see that:
\begin{align*}
    \beta_1 &= \phi \\
    \beta_2 &= \hat{\alpha}_1 - \hat{\alpha}_2 \phi \\
    \mu &= \xi
\end{align*}

### 2. Show that the Conditional Expectation Function minimizes expected squared error

Given the model:
\begin{align*}
Y &= m(X) + e
\end{align*}
where $m(X)$ represents the conditional expectation of $Y$ on $X$. Let's define an arbitrary model:
\begin{align*}
Y &= g(X) + w
\end{align*}
where $g(X)$ represents any function of $X$.

Working with the expected squared error from the arbitrary model:
\begin{align*}
E[(Y-g(X))^2] &= E[(Y-m(X) + m(X)-g(X))^2] \\
&= E[(Y-m(X))^2 + 2(Y-m(X))(m(X)-g(X)) + (m(X)-g(X))^2] \\
&= E[e^2] + 2E[(Y-m(X))(m(X)-g(X))] + E[(m(X)-g(X))^2]
\end{align*}
Using the law of iterated expectations:
\begin{align*}
E[(Y-g(X))^2] &= E[e^2] + 2E[E[(Y-m(X))(m(X)-g(X)) | X]] + E[(m(X)-g(X))^2]
\end{align*}
Since $m(X)$ and $g(X)$ are functions of $X$, the term $(m(X)-g(X))$ can be thought of as constant when conditioning on $X$. Thus:
\begin{align*}
E[(Y-g(X))^2] &= E[e^2] + 2E[E[Y-m(X) | X](m(X)-g(X))] + E[(m(X)-g(X))^2]
\end{align*}
It is important to note that $E[Y-m(X) | X] = 0$ by definition of $m(X)$, so we get:
\begin{align*}
E[(Y-g(X))^2] &= E[e^2] + E[(m(X)-g(X))^2]
\end{align*}
Because the second term in the equation is always non-negative, it is clear that the function is minimized when $g(X)$ equals $m(X)$. In which case:
\begin{align*}
E[(Y-g(X))^2] &= E[e^2]
\end{align*}

This notebook contains an example for teaching.

## **Replication 1 - Code**

In the previous lab, we already analyzed data from the March Supplement of the U.S. Current Population Survey (2015) and answered the question how to use job-relevant characteristics, such as education and experience, to best predict wages. Now, we focus on the following inference question:

What is the difference in predicted wages between men and women with the same job-relevant characteristics?

Thus, we analyze if there is a difference in the payment of men and women (*gender wage gap*). The gender wage gap may partly reflect *discrimination* against women in the labor market or may partly reflect a *selection effect*, namely that women are relatively more likely to take on occupations that pay somewhat less (for example, school teaching).

To investigate the gender wage gap, we consider the following log-linear regression model

\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 D  + \beta_2' W + \epsilon,
\end{align}

where $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the
$W$'s are controls explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of men and women.

### Data Analysis

We consider the same subsample of the U.S. Current Population Survey (2015) as in the previous lab. Let us load the data set.



In [1]:
using Pkg

Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
Pkg.add("CategoricalArrays")

using DataFrames
using Dates
using Plots
using Statistics,RData  #upload data of R format 
using CategoricalArrays # categorical data 

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


LoadError: ArgumentError: Package RData not found in current path.
- Run `import Pkg; Pkg.add("RData")` to install the RData package.

In [2]:
rdata_read = load("../../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In [3]:
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,wage,23.4104,3.02198,19.2308,528.846,0,Float64
2,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
3,sex,0.444466,0.0,0.0,1.0,0,Float64
4,shs,0.023301,0.0,0.0,1.0,0,Float64
5,hsg,0.243883,0.0,0.0,1.0,0,Float64
6,scl,0.278058,0.0,0.0,1.0,0,Float64
7,clg,0.31767,0.0,0.0,1.0,0,Float64
8,ad,0.137087,0.0,0.0,1.0,0,Float64
9,mw,0.259612,0.0,0.0,1.0,0,Float64


To start our (causal) analysis, we compare the sample means given gender:

In [4]:
Z = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"])

data_female = filter(row -> row.sex == 1, data)
Z_female = select(data_female,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

data_male = filter(row -> row.sex == 0, data)
Z_male = select(data_male,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

means = DataFrame( variables = names(Z), All = describe(Z, :mean)[!,2], Men = describe(Z_male,:mean)[!,2], Female = describe(Z_female,:mean)[!,2])


,variables,All,Men,Female
,String,Float64,Float64,Float64
1,lwage,2.97079,2.98783,2.94948
2,sex,0.444466,0.0,1.0
3,shs,0.023301,0.0318071,0.0126693
4,hsg,0.243883,0.294303,0.180865
5,scl,0.278058,0.273331,0.283967
6,clg,0.31767,0.293953,0.347313
7,ad,0.137087,0.106606,0.175186
8,ne,0.227767,0.22195,0.235037
9,mw,0.259612,0.259,0.260376


In particular, the table above shows that the difference in average logwage between men and women is equal to $0,038$

In [5]:
mean(Z_female[:,:lwage]) - mean(Z_male[:,:lwage])

-0.03834473367441493

Thus, the unconditional gender wage gap is about $3,8$\% for the group of never married workers (women get paid less on average in our sample). We also observe that never married working women are relatively more educated than working men and have lower working experience.

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in Julia.

In [6]:
#install all the package that we can need
Pkg.add("GLM") # package to run models 
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("Tables")
Pkg.add("CovarianceMatrices") # robust standar error 
# Load the installed packages
using DataFrames
using CSV
using Tables
using GLM
using CovarianceMatrices


   Resolving package versions...
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed MLBase ─ v0.9.2
    Updating `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  [f0e99cf1] + MLBase v0.9.2
    Updating `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
  [f0e99cf1] ↑ MLBase v0.9.0 ⇒ v0.9.2
Precompiling project...
  ✓ MLBase
  ✓ Lasso
  2 dependencies successfully precompiled in 4 seconds (323 already precompiled)
   Resolving package versions...
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to

In [7]:
nocontrol_model = lm(@formula(lwage ~ sex), data)
nocontrol_est = GLM.coef(nocontrol_model)[2]
nocontrol_se = GLM.coeftable(nocontrol_model).cols[2][2]

nocontrol_se1 = stderror(HC1(), nocontrol_model)[2]
println("The estimated gender coefficient is ", nocontrol_est ," and the corresponding robust standard error is " ,nocontrol_se1)

The estimated gender coefficient is -0.03834473367440746 and the corresponding robust standard error is 0.015905023733117172


Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

### Ols regression with controls

In [8]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
control_model = lm(flex , data)
control_est = GLM.coef(control_model)[2]
control_se = GLM.coeftable(control_model).cols[2][2]
control_se1 = stderror( HC0(), control_model)[2]


0.015000474421752112

In [9]:
control_model 

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────

In [10]:
println("Coefficient for OLS with controls " , control_est, "robust standard error:", control_se1)

Coefficient for OLS with controls -0.06955320329637253robust standard error:0.015000474421752112


The estimated regression coefficient $\beta_1\approx-0.0696$ measures how our linear prediction of wage changes if we set the gender variable $D$ from 0 to 1, holding the controls $W$ fixed.
We can call this the *predictive effect* (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size $4$\% for women increases to about $7$\% after controlling for worker characteristics.  


Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

### Partialling-Out using ols

In [11]:
# models
# model for Y
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))

# partialling-out the linear effect of W from Y
t_Y = residuals(lm(flex_y, data))

# partialling-out the linear effect of W from D
t_D = residuals(lm(flex_d, data))

data_res = DataFrame(t_Y = t_Y, t_D = t_D )
# regression of Y on D after partialling-out the effect of W

partial_fit = lm(@formula(t_Y ~ t_D), data_res)

partial_est = GLM.coef(partial_fit)[2]

# standard error
partial_se = GLM.coeftable(partial_fit).cols[2][2]

partial_se1 = stderror( HC0(), partial_fit)[2]

#condifence interval
GLM.confint(partial_fit)[2,:]

2-element Vector{Float64}:
 -0.09867142357486275
 -0.04043498301882939

In [12]:
println("Coefficient for D via partiallig-out ", partial_est, " robust standard error:", control_se1 )

Coefficient for D via partiallig-out -0.06955320329684607 robust standard error:0.015000474421752112


Again, the estimated coefficient measures the linear predictive effect (PE) of $D$ on $Y$ after taking out the linear effect of $W$ on both of these variables. This coefficient equals the estimated coefficient from the ols regression with controls.

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 

In the following, we illustrate the partialling-out approach using lasso instead of ols. 

### Summarize the results

In [13]:
DataFrame(modelos = [ "Without controls", "full reg", "partial reg" ], 
Estimate = [nocontrol_est,control_est, partial_est], 
StdError = [nocontrol_se1,control_se1, partial_se1])

,modelos,Estimate,StdError
,String,Float64,Float64
1,Without controls,-0.0383447,0.015905
2,full reg,-0.0695532,0.0150005
3,partial reg,-0.0695532,0.0150005


It it worth to notice that controlling for worker characteristics increases the gender wage gap from less that 4\% to 7\%. The controls we used in our analysis include 5 educational attainment indicators (less than high school graduates, high school graduates, some college, college graduate, and advanced degree), 4 region indicators (midwest, south, west, and northeast);  a quartic term (first, second, third, and fourth power) in experience and 22 occupation and 23 industry indicators.

Keep in mind that the predictive effect (PE) does not only measures discrimination (causal effect of being female), it also may reflect
selection effects of unobserved differences in covariates between men and women in our sample.


Next we try "extra" flexible model, where we take interactions of all controls, giving us about 1000 controls.

### "Extra" flexible model

In [14]:
import Pkg
Pkg.add("StatsModels")
Pkg.add("Combinatorics")
Pkg.add("IterTools")
# we have to configure the package internaly with the itertools package, this because 
#julia dont iunderstand (a formula) ^2, it takes as an entire term not as interactions 
#between variables

   Resolving package versions...
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  [861a8166] + Combinatorics v1.0.2
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\Alexander\.julia\environments\v1.7\Project.toml`
  [c8e1da08] + IterTools v1.4.0
  No Changes to `C:\Users\Alexander\.julia\environments\v1.7\Manifest.toml`


In [15]:
#this code fix the problem mencioned above
using StatsModels, Combinatorics, IterTools

combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)
expand_exp(args, deg::ConstantTerm) =
    tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.FullRank, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [16]:
extra_flex = @formula(lwage ~  sex + (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2)

control_fit = lm(extra_flex, data)
control_est = GLM.coef(control_fit)[2]

println("Number of Extra-Flex Controls: ", size(modelmatrix(control_fit))[2] -1) #minus the intercept
println("Coefficient for OLS with extra flex controls ", control_est)

#std error
control_se = GLM.stderror(control_fit)[2];

Number of Extra-Flex Controls: 979
Coefficient for OLS with extra flex controls -0.061270463792332606


## **Cross-Validation in Lasso Regression - Manual Implementation Task**

In [94]:
# import Pkg; Pkg.add("ScikitLearn")
# import Pkg; Pkg.add("StatsBase")
# import Pkg; Pkg.add("CSVFiles")
# import Pkg; Pkg.add("RDatasets")
# import Pkg; Pkg.add("MLJ")

LoadError: The following package names could not be resolved:
 * StandardScaler (not found in project, manifest or registry)
[36m   Suggestions:[39m [0m[1mS[22m[0m[1mt[22m[0m[1ma[22m[0m[1mn[22m[0m[1md[22m[0m[1ma[22m[0m[1mr[22m[0m[1md[22mizedRe[0m[1ms[22mtri[0m[1mc[22mtedBoltzm[0m[1ma[22mnnMachines

In [100]:
using Pkg
Pkg.add("Transformers")

   Resolving package versions...
   Installed ShowCases ──────────── v0.1.0
   Installed ContextVariablesX ──── v0.1.3
   Installed Accessors ──────────── v0.1.36
   Installed ZipFile ────────────── v0.9.4
   Installed HTML_Entities ──────── v1.0.1
   Installed TimerOutputs ───────── v0.5.23
   Installed NNlibCUDA ──────────── v0.2.6
   Installed NNlib ──────────────── v0.8.21
   Installed Pidfile ────────────── v1.3.0
   Installed Optimisers ─────────── v0.2.15
   Installed FuncPipelines ──────── v0.2.3
   Installed InitialValues ──────── v0.3.1
   Installed CEnum ──────────────── v0.4.2
   Installed FunctionWrappers ───── v1.1.3
   Installed BFloat16s ──────────── v0.2.0
   Installed GPUArrays ──────────── v8.8.1
   Installed ZygoteRules ────────── v0.2.5
   Installed PrettyPrint ────────── v0.2.0
   Installed HuggingFaceApi ─────── v0.1.0
   Installed Static ─────────────── v0.7.8
   Installed RandomNumbers ──────── v1.5.3
   Installed Tricks ─────────────── v0.1.8
   Installed Real

In [101]:
using Pkg
using RData
using DataFrames
using RDatasets
using Random
using StatsBase
using GLM
using PyCall
using Statistics
using Plots
using CSV
using Transformers

import PyPlot
import DataFrames
import StatsModels
import Random
import ScikitLearn: cross_val_score

Precompiling Transformers
  ✗ BinaryProvider
  0 dependencies successfully precompiled in 2 seconds. 183 already precompiled.
[ Info: Precompiling Transformers [21ca0261-441d-5938-ace7-c90938fde4d4]
┌ Warning: Module HTTP with build ID fafbfcfd-fb7e-58a8-0000-0286b56ffd08 is missing from the cache.
│ This may mean HTTP [cd3eb016-35fb-5094-929b-558a96fad6f3] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing Transformers [21ca0261-441d-5938-ace7-c90938fde4d4].
[ Info: Precompiling ZygoteColorsExt [e68c091a-8ea5-5ca7-be4f-380657d4ad79]
┌ Warning: Module Colors with build ID fafbfcfd-6f69-f41e-0000-2644de38854d is missing from the cache.
│ This may mean Colors [5ae59095-9a9b-59fe-a467-6f913c188581] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing ZygoteColors

**1. Data Preparation**

Load the March Supplement of the U.S. Current Population Survey, year 2015. (wage2015_subsample_inference.Rdata)

In [80]:
rdata_read = RData.load("../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)

20-element Vector{String}:
 "wage"
 "lwage"
 "sex"
 "shs"
 "hsg"
 "scl"
 "clg"
 "ad"
 "mw"
 "so"
 "we"
 "ne"
 "exp1"
 "exp2"
 "exp3"
 "exp4"
 "occ"
 "occ2"
 "ind"
 "ind2"

In [81]:
# a quick description of the data
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,wage,23.4104,3.02198,19.2308,528.846,0,Float64
2,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
3,sex,0.444466,0.0,0.0,1.0,0,Float64
4,shs,0.023301,0.0,0.0,1.0,0,Float64
5,hsg,0.243883,0.0,0.0,1.0,0,Float64
6,scl,0.278058,0.0,0.0,1.0,0,Float64
7,clg,0.31767,0.0,0.0,1.0,0,Float64
8,ad,0.137087,0.0,0.0,1.0,0,Float64
9,mw,0.259612,0.0,0.0,1.0,0,Float64


In [82]:
# flexible model
# flex = @formula(lwage ~ sex + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + occ2 + ind2 + mw + so + we))
# flex_results_0 = lm(flex, data)

In [83]:
# # Get exogenous variables (X matrix) from the flexible model
# X = Matrix(modelmatrix(flex_results_0))

# # Set endogenous variable (response variable)
# y = data[!, "lwage"]

In [84]:
function partitionTrainTest(data, at = 0.7)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    data[train_idx,:], data[test_idx,:]
end

iris = dataset("datasets", "iris")
X_train,X_test = partitionTrainTest(iris, 0.7) # 70% train

(105×5 DataFrame
 Row │ SepalLength  SepalWidth  PetalLength  PetalWidth  Species    
     │ Float64      Float64     Float64      Float64     Cat…       
─────┼──────────────────────────────────────────────────────────────
   1 │         6.1         3.0          4.9         1.8  virginica
   2 │         5.5         2.6          4.4         1.2  versicolor
   3 │         5.6         2.7          4.2         1.3  versicolor
   4 │         5.3         3.7          1.5         0.2  setosa
   5 │         6.2         2.2          4.5         1.5  versicolor
   6 │         7.7         3.8          6.7         2.2  virginica
   7 │         6.5         2.8          4.6         1.5  versicolor
   8 │         6.6         2.9          4.6         1.3  versicolor
   9 │         5.5         3.5          1.3         0.2  setosa
  10 │         6.0         2.9          4.5         1.5  versicolor
  11 │         5.5         2.3          4.0         1.3  versicolor
  ⋮  │      ⋮           ⋮            ⋮

In [102]:
scaler = StandardScaler()

# Fit and transform on training data
X_train = MLJ.transform(scaler, X_train)

# Transform test data using the same scaler
X_test = MLJ.transform(scaler, X_test)

LoadError: UndefVarError: `StandardScaler` not defined

**2. Define a Range of Alpha (Lambda in our equation) Values**

We create a list or array of alpha values to iterate over. These will be the different regularization parameters we test. We started testing from 0.1 to 0.5 and found that the MSE in cross-validation was reducing when the alpha value was incrementing. Therefore, we tried with higher values.

In [ ]:
# Create alphas using a loop
alphas = [45 + i * 5 for i in 0:4]

**3. Partition the Dataset for k-Fold Cross-Validation**

We divide the dataset into 5 subsets (or folds). Since we are working with a regression task (predicting the log of wage), we use the K-Fold cross-validator from sklearn. We ensure the data is shuffled by adding 'shuffle=True' and set a random state for a reproducible output.

Source: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [ ]:
# Create KFold
kf = KFold(n_splits = 5, shuffle = true, random_state = 24)

**4. Lasso Regression Implementation**

Implement a function to fit a Lasso Regression model given a training dataset and an alpha value. The function should return the model's coefficients and intercept.

In [ ]:
function lasso_regression(X_train, y_train, alpha, iterations=100, learning_rate=0.01)
    """
    Fits a Lasso Regression model

    Args:
        X_train: Training features
        y_train: Target values
        alpha: Regularization parameter (L1 penalty)
        iterations: Number of iterations for gradient descent (default: 100)
        learning_rate: Learning rate for gradient descent (default: 0.01)

    Returns:
        W: Model coefficients (weights)
        b: Model intercept (bias)
    """
    m, n = size(X_train)
    W = zeros(n)
    b = 0.0

    for _ in 1:iterations
        Y_pred = X_train * W .+ b
        dW = zeros(n)
        for j in 1:n
            if W[j] > 0
                dW[j] = (-2 * dot(X_train[:, j], y_train - Y_pred) + alpha) / m
            else
                dW[j] = (-2 * dot(X_train[:, j], y_train - Y_pred) - alpha) / m
            end
        end
        db = -2 * sum(y_train - Y_pred) / m
        W .-= learning_rate * dW
        b -= learning_rate * db
    end

    return W, b
end

**5. Cross-Validation Loop and 6. Selection of Optimal Alpha**

We immplement a for loop to fit the lasso regression. Also, we find the best value of alpha that reduces the average MSE for each fold.

In [ ]:
function cross_validate_lasso(X_train, y_train, alphas, kf)
    avg_mse_values = Float64[]
    best_alpha = nothing
    min_avg_mse = Inf

    for alpha in alphas
        mse_list = Float64[]
        for (train_index, val_index) in kf
            X_train_fold, X_val_fold = X_train[train_index, :], X_train[val_index, :]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            # Train Lasso regression model with the current alpha
            W, b = lasso_regression(X_train_fold, y_train_fold, alpha)
            y_pred_val = X_val_fold * W .+ b

            # Calculate MSE for this fold
            mse_fold = mean((y_val_fold .- y_pred_val).^2)
            push!(mse_list, mse_fold)
        end

        # Calculate average MSE across all folds
        avg_mse = mean(mse_list)
        push!(avg_mse_values, avg_mse)
        println("Alpha=$(round(alpha, digits=1)), Average MSE: $(round(avg_mse, digits=5))")

        # Update the best alpha and minimum average MSE
        if avg_mse < min_avg_mse
            min_avg_mse = avg_mse
            best_alpha = alpha
        end
    end

    println("Best Alpha: $(round(best_alpha, digits=1)), Minimum Average MSE: $(round(min_avg_mse, digits=5))")

    # Plotting the cross-validated MSE for each alpha value
    plt = plot(alphas, avg_mse_values, marker = :o, linestyle = :solid, xlabel = "Alpha", ylabel = "Average MSE",
               title = "Cross-Validated MSE for Different Alpha Values", legend = false)
    xticks!(plt, alphas)
    grid!(plt, true)
    display(plt)
end

**7. Model Training and Evaluation**

In [ ]:
# Train Lasso regression model with the best alpha
W, b = lasso_regression(X_train, y_train, best_alpha)

# Make predictions on test data
y_pred = X_test * W .+ b

In [ ]:
lasso_corr = cor(y_test, y_pred)
lasso_mae = mean(abs.(y_test .- y_pred))
lasso_mse = mean((y_test .- y_pred).^2)

# Print results
println("Correlation: $(round(lasso_corr, digits=4))")
println("MAE: $(round(lasso_mae, digits=4))")
println("MSE: $(round(lasso_mse, digits=4))")

**8. Report Results**

We began by selecting the parameters for a flexible model, one that includes interactions. After selecting the parameters, we found that the alpha value that produced the lowest mean squared error (MSE) in cross-validation was 60. We then trained the model using all the available training data with the best alpha value. Using the fitted betas, we predicted the lwage using the test X vector. The resulting MSE in the test data was 0.3569, which was lower than in the training data (which was 0.41906). This indicates that the model is not overfitting and that we have found a good correlation score (R square) of 50%.

# Replication

In [ ]:
Wage

In [ ]:
using Statistics
using Plots


wage_sorted = sort(data.wage)
cumulative_sum = cumsum(wage_sorted)
deciles = [percentile(cumulative_sum, p) for p in 0:10:100]
population_fraction = collect(0:0.1:1)

# Calcular la fracción acumulativa de salarios
salary_fraction = [percentile(deciles ./ sum(data.wage), p) for p in 0:10:100]

# Graficar la curva de Lorenz
plot(population_fraction, salary_fraction, linewidth=2, color=:blue, label="")
plot!([0, 1], [0, 1], linestyle=:dash, linewidth=1, color=:red, label="Equality line",
    title="Lorenz Curve for Higher Education Salaries",
    xlabel="Cumulative fraction of the population",
    ylabel="Cumulative fraction of salaries",
    legend=:bottomright)


In [ ]:
LwAGE

In [ ]:
using Plots
using StatsPlots

# Configurar el estilo
default(; legend = false)

# Crear el histograma de los salarios
histogram(data.lwage, bins=30, color=:lightblue, edgecolor=:black, normed=true, xlabel="Lwage", ylabel="Density",
    title="Histogram of Salaries")

# Crear la función de densidad
density!(data.lwage, color=:red, linewidth=1)



In [ ]:
Sex

In [ ]:
import Pkg
Pkg.add("PyPlot")
Pkg.add("Distributions")


In [ ]:
using DataFrames
using Statistics
using Plots

# Calcular las proporciones de sexos
proportions = combine(groupby(data, :sex), nrow => :count)

# Calcular el total de observaciones
total_obs = sum(proportions.count)

# Asignar etiquetas 'Male' y 'Female'
proportions[!, :sex_label] = ifelse.(proportions.sex .== 0, "Male", "Female")

# Calcular las proporciones
proportions[!, :percentage] = proportions.count ./ total_obs * 100

# Graficar las proporciones sin leyenda
display(bar(proportions.sex_label, proportions[!, :percentage],
    xlabel = "Sex", ylabel = "Percentage",
    title = "Proportion of Males and Females",
    ylims = (0, 100),
    bar_width = 0.5,
    fmt = :png))

# Mostrar el número total de observaciones
println("Total observations: ", total_obs)


In [ ]:
Dummies college

In [ ]:
using DataFrames
using Statistics
using Plots

# Crear la variable 'Education_Status'
data[!, "Education_Status"] .= ifelse.(data.scl .== 1, "Some College",
                                      ifelse.(data.clg .== 1, "College Graduate", "Advanced Degree"))

# Calcular la frecuencia de educación
edu_freq = combine(groupby(data, :Education_Status), nrow => :Frequency)

# Calcular el total de observaciones
total_obs = sum(edu_freq.Frequency)

# Calcular el porcentaje
edu_freq[!, :Percentage] = edu_freq.Frequency / total_obs * 100

# Definir colores y etiquetas
# Crear el gráfico de pastel
pie(edu_freq.Frequency, labels=string.(labels, ": ", round.(edu_freq.Percentage, digits=1), "%"), startangle=90, 
    title="Education Status Distribution", 
    legend=:topright, 
    fmt=:png, 
    aspect_ratio=1,
    pct=true)

# Añadir el número total de observaciones
annotate!(1, -1.2, text("Total observations: $total_obs", halign=:center, valign=:center))


In [ ]:
Experience Higher Education

In [ ]:
using DataFrames
using Plots

plt = plot(size=(800, 600))
boxplot!(data.exp1, box=:box, whiskerwidth=0.2, title="Distribution of Experience in Individuals with Higher Education",
    ylabel="Experience (exp1)", legend=false)


In [ ]:
COEFFICIENTS FOR DIFFERENT MODELS

In [ ]:
using DataFrames
using GLM
using Statistics
using Plots


# Ajustar los modelos
flex_y = @formula(lwage ~ exp1+exp2+exp3+exp4+scl+clg+ad+occ2+ind2+mw+so+we+(exp1+exp2+exp3+exp4)*(scl+clg+ad+occ2+ind2+mw+so+we) ) # modelo para Y
flex_d = @formula(sex ~ exp1+exp2+exp3+exp4+scl+clg+ad+occ2+ind2+mw+so+we+(exp1+exp2+exp3+exp4)*(scl+clg+ad+occ2+ind2+mw+so+we))  # modelo para D

# Ajustar el efecto lineal parcial de W desde Y
t_Y = residuals(lm(flex_y, data))
# Ajustar el efecto lineal parcial de W desde D
t_D = residuals(lm(flex_d, data))
# Crear un nuevo DataFrame con las variables relevantes para t_Y y t_D
residuals_df = DataFrame(t_Y=t_Y, t_D=t_D)

# Ajustar la regresión lineal entre t_Y y t_D
partial_fit = lm(@formula(t_D ~ t_Y), residuals_df)

# Ajustar los modelos
basic_fit = lm(@formula(lwage ~ sex + exp1 + scl + clg + ad + mw + so + we + occ2 + ind2), data)

control_fit = lm(@formula(lwage ~ sex + exp1+exp2+exp3+exp4+scl+clg+ad+occ2+ind2+mw+so+we+(exp1+exp2+exp3+exp4)*(scl+clg+ad+occ2+ind2+mw+so+we)), data)

using Plots

# Extraer los coeficientes para sex y los errores estándar de las últimas dos regresiones
coefs = Dict("Basic" => coef(basic_fit)[2],
             "Controls" => coef(control_fit)[2],
             "Partialling out" => coef(partial_fit)[2])  # El coeficiente de interés es el segundo en partial_fit

ses = Dict("Basic" => stderror(basic_fit)[2],
           "Controls" => stderror(control_fit)[2],
           "Partialling out" => stderror(partial_fit)[2])  # El error estándar correspondiente al coeficiente de interés

# Gráfico de dispersión con barras de error
scatter_plot = scatter(coefs, yerr=1.96 .* collect(values(ses)), legend=false, 
    xlabel="Model", ylabel="Coefficient", title="Effect of sex on lwage",
    markershape=:circle, markercolor=:blue, markerstrokecolor=:black,
    markersize=8, markeralpha=0.8)

# Línea horizontal en y=0
hline!(scatter_plot, [0], color="gray", linestyle=:dash)

# Mostrar el gráfico
display(scatter_plot)


In [ ]:
#The coefficient associated with the gender variable, which indicates the prediction of being female on salary, is initially 
# negative. This suggests that, on average, women have lower salaries than men. However, after adding these controls, 
# such as work experience or educational level, the negative coefficient associated with the gender variable becomes 
# less negative.
# 
# This change in the gender coefficient could be explained by the fact that the control variables are capturing
# some of the variability in salaries that was previously incorrectly attributed to gender. This suggests 
# that additional factors, beyond gender, are influencing salaries, and the impact of gender on salaries 
# is less pronounced once these other variables are taken into account.Besides, both FWL and including control 
# variables in the regression model yield coefficient estimates for the variable of interest that reflect its net
# impact on the dependent variable, once the effects of other explanatory variables have been taken into account.


# Women and male graph (quartic)

In [ ]:
import Pkg
Pkg.add("StatsModels")


In [ ]:
import Pkg
Pkg.add("Loess")


In [ ]:
using DataFrames
using Statistics
using Plots

# Supongamos que tienes un DataFrame llamado data con las columnas lwage, exp4 y sex

# Creamos un DataFrame separado para hombres y mujeres
data_male = filter(row -> row.sex == 0, data)


# Calculamos las medias de lwage para cada valor único de exp4 para hombres
mean_lwage_male = combine(groupby(data_male, :exp4), :lwage => mean => :mean_lwage)

# Ordenar el DataFrame por exp4
sort!(mean_lwage_male, :exp4)

# Crear el gráfico para hombres
plot(mean_lwage_male.exp4, mean_lwage_male.mean_lwage, color=:blue, label="Mean lwage (Male)",
    xlabel="Experiencia (exp4)", ylabel="Media de lwage (lwage)", title="Media de lwage por exp4 (Hombres)", marker=:circle)
plot!(mean_lwage_male.exp4, mean_lwage_male.mean_lwage, color=:blue, linewidth=2, linestyle=:solid, label="")


In [ ]:
using Pkg
Pkg.add("Loess")


In [ ]:
using DataFrames
using Plots
using Loess

# Supongamos que tienes un DataFrame llamado data con las columnas lwage, exp4 y sex

# Creamos un DataFrame separado solo para varones
data_male = filter(row -> row.sex == 0, data)

# Extraemos las variables independientes y dependientes para el modelo LOESS
exp4_male = data_male.exp4
lwage_male = data_male.lwage

# Ajustamos un modelo LOESS para varones con un span de ** para menos suavizamiento
loess_model_male = loess(exp4_male, lwage_male, span=0.95)

# Generamos predicciones para el modelo LOESS
exp4_range_male = range(minimum(exp4_male), maximum(exp4_male), length=500)
lwage_pred_male = predict(loess_model_male, exp4_range_male)

# Creamos el gráfico para varones con la línea suavizada
plot(exp4_range_male, lwage_pred_male, color=:red, label="Smoothed lwage (Male)")
xlabel!("Experiencia (exp4)")
ylabel!("lwage")
title!("Línea suavizada de lwage por exp4 (Varones)")
xlims!(0, 40)  # Limitar el eje x de 0 a 40


In [ ]:
data_female = filter(row -> row.sex == 1, data)
# Calculamos las medias de lwage para cada valor único de exp4 para mujeres
mean_lwage_female = combine(groupby(data_female, :exp4), :lwage => mean => :mean_lwage)

# Ordenar el DataFrame por exp4
sort!(mean_lwage_female, :exp4)

# Crear el gráfico para mujeres
plot(mean_lwage_female.exp4, mean_lwage_female.mean_lwage, color=:red, label="Mean lwage (Female)",
    xlabel="Experiencia (exp4)", ylabel="Media de lwage (lwage)", title="Media de lwage por exp4 (Mujeres)", marker=:circle)
plot!(mean_lwage_female.exp4, mean_lwage_female.mean_lwage, color=:red, linewidth=2, linestyle=:solid, label="")


In [ ]:
using DataFrames
using Plots
using Loess

# Supongamos que tienes un DataFrame llamado data con las columnas lwage, exp4 y sex

# Creamos un DataFrame separado solo para mujeres
data_female = filter(row -> row.sex == 1, data)

# Extraemos las variables independientes y dependientes para el modelo LOESS
exp4_female = data_female.exp4
lwage_female = data_female.lwage

# Ajustamos un modelo LOESS para mujeres con un span de 0.95 para menos suavizamiento
loess_model_female = loess(exp4_female, lwage_female, span=0.95)

# Generamos predicciones para el modelo LOESS
exp4_range_female = range(minimum(exp4_female), maximum(exp4_female), length=500)
lwage_pred_female = predict(loess_model_female, exp4_range_female)

# Creamos el gráfico para mujeres con la línea suavizada
plot(exp4_range_female, lwage_pred_female, color=:red, label="Smoothed lwage (Female)")
xlabel!("Experiencia (exp4)")
ylabel!("lwage")
title!("Línea suavizada de lwage por exp4 (Mujeres)")
xlims!(0, 40)  # Limitar el eje x de 0 a 40


In [ ]:
using DataFrames
using Plots
using Loess

# Supongamos que tienes un DataFrame llamado data con las columnas lwage, exp4 y sex

# Creamos un DataFrame separado solo para mujeres
data_female = filter(row -> row.sex == 1, data)

# Calculamos las medias de lwage para cada valor único de exp4 para mujeres
mean_lwage_female = combine(groupby(data_female, :exp4), :lwage => mean => :mean_lwage)

# Ordenar el DataFrame por exp4
sort!(mean_lwage_female, :exp4)

# Extraemos las variables independientes y dependientes para el modelo LOESS
exp4_female = data_female.exp4
lwage_female = data_female.lwage

# Ajustamos un modelo LOESS para mujeres con un span de 0.95 para menos suavizamiento
loess_model_female = loess(exp4_female, lwage_female, span=0.95)

# Generamos predicciones para el modelo LOESS
exp4_range_female = range(minimum(exp4_female), maximum(exp4_female), length=500)
lwage_pred_female = predict(loess_model_female, exp4_range_female)

# Creamos el gráfico para mujeres con la línea suavizada y los puntos unidos
plot(exp4_range_female, lwage_pred_female, color=:red, label="Smoothed lwage (Female)", linewidth=2)
scatter!(mean_lwage_female.exp4, mean_lwage_female.mean_lwage, color=:red, label="", marker=:circle, seriestype=:line)
xlabel!("Experiencia (exp4)")
ylabel!("lwage")
title!("Media de lwage y línea suavizada por exp4 (Mujeres)")
xlims!(0, 40)  # Limitar el eje x de 0 a 40
